In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
from PIL import Image
import datetime
import openpyxl

In [119]:
# Подключаем api_key из файла
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()

In [120]:
# Составляем URL request
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2020-01-01'
end_date = '2020-01-01'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [121]:
'''
Функция навигации по данным из api response   
возвращает ссылку на данные в другом временном отрезке

    step = 'next' -- следующий день от заданного start_date в URL request
    step = 'prev' -- предыдущий день от заданного start_date в URL request
    step = 'self' -- текущий день, равный start_date в URL request
'''

def get_url(url, step):
    url_level = urllib.request.urlopen(url)
    data_level = json.loads(url_level.read())['links']
    if step == 'next':
        return data_level[step]
    elif step == 'prev':
        return data_level[step]
    elif step == 'self':
        return data_level[step]

In [122]:
'''
Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)
и вывод даты в строку

delta = datetime.timedelta(days=1)
day_current = datetime.date.fromisoformat(start_date) + delta
date_str = datetime.date.strftime(day_current, '%Y-%m-%d')
'''

"\nВариант использования бибилотеки datetime для скольжения по датам (не используется в коде)\nи вывод даты в строку\n\ndelta = datetime.timedelta(days=1)\nday_current = datetime.date.fromisoformat(start_date) + delta\ndate_str = datetime.date.strftime(day_current, '%Y-%m-%d')\n"

In [123]:
'''Навигация по ссылкам.

Работаем с данными за один день 2020-07-07.
Обрабатываем входящий json.
'''

url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

,links,element_count,near_earth_objects
next,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,13,NaN
prev,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,13,NaN
self,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,13,NaN
2020-01-01,NaN,13,[{'links': {'self': 'http://api.nasa.gov/neo/r...


In [124]:
# Выбираем один элемент из списка за один день
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3564720?api_key=8Mbg6rxSagXNYxrUbC8ynidJUcLn9h4CC7ucuT21'},
 'id': '3564720',
 'neo_reference_id': '3564720',
 'name': '(2011 HS60)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3564720',
 'absolute_magnitude_h': 21.3,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.1460679643,
   'estimated_diameter_max': 0.3266178974},
  'meters': {'estimated_diameter_min': 146.0679642714,
   'estimated_diameter_max': 326.6178974458},
  'miles': {'estimated_diameter_min': 0.090762397,
   'estimated_diameter_max': 0.2029508896},
  'feet': {'estimated_diameter_min': 479.2256199,
   'estimated_diameter_max': 1071.581062656}},
 'is_potentially_hazardous_asteroid': False,
 'close_approach_data': [{'close_approach_date': '2020-01-01',
   'close_approach_date_full': '2020-Jan-01 21:59',
   'epoch_date_close_approach': 1577915940000,
   'relative_velocity': {'kilometers_per_second': '17.7744593888',
    'kilometers_per_hou

In [125]:
# Создаем словари с необходимями данными для DataFrame
dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
dict_close_approach_data = data_day_item['close_approach_data'][0]
dict_relative_velocity = dict_close_approach_data['relative_velocity']
dict_miss_distance = dict_close_approach_data['miss_distance']

In [126]:
# Чистим словарь от вложенных словарей
keys = ['links','estimated_diameter','close_approach_data', 'relative_velocity', 'miss_distance']

for key in keys:
    data_day_item.pop(key, None)
data_day_item_short = data_day_item

data_day_item_short

{'id': '3564720',
 'neo_reference_id': '3564720',
 'name': '(2011 HS60)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3564720',
 'absolute_magnitude_h': 21.3,
 'is_potentially_hazardous_asteroid': False,
 'is_sentry_object': False}

In [127]:
'''Создаем DataFrame для выбранных данных.

Объединяем DataFrame в один для дальнейшей работы.
'''

df1 = pd.DataFrame([data_day_item_short])
df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)
df_item = df1.join(df2).join(df3).join(df4).join(df5)

df_item

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_min,estimated_diameter_max,close_approach_date,...,relative_velocity,miss_distance,orbiting_body,kilometers_per_second,kilometers_per_hour,miles_per_hour,astronomical,lunar,kilometers,miles
0,3564720,3564720,(2011 HS60),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3564720,21.3,False,False,0.146068,0.326618,2020-01-01,...,NaN,NaN,Earth,17.7744593888,63988.0537997394,39759.6882068436,0.198878634,77.363788626,29751820.03490958,18486923.719649004


In [128]:
# Чистим DataFrame от нерелевантных данных, переименовываем
df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                'neo_reference_id',
                                                'miss_distance',
                                                'miles_per_hour',
                                                'astronomical',
                                                'lunar',
                                                'miles'
                                                ], axis=1
).rename(columns={'kilometers': 'miss_distance_kilometers',
                'estimated_diameter_min': 'estimated_diameter_km_min',
                'estimated_diameter_max': 'estimated_diameter_km_max'})

# Создаем словарь из DataFrame
df_item.to_dict()

{'id': {0: '3564720'},
 'name': {0: '(2011 HS60)'},
 'absolute_magnitude_h': {0: 21.3},
 'is_potentially_hazardous_asteroid': {0: False},
 'is_sentry_object': {0: False},
 'estimated_diameter_km_min': {0: 0.1460679643},
 'estimated_diameter_km_max': {0: 0.3266178974},
 'close_approach_date': {0: '2020-01-01'},
 'close_approach_date_full': {0: '2020-Jan-01 21:59'},
 'epoch_date_close_approach': {0: 1577915940000},
 'relative_velocity': {0: nan},
 'orbiting_body': {0: 'Earth'},
 'kilometers_per_second': {0: '17.7744593888'},
 'kilometers_per_hour': {0: '63988.0537997394'},
 'miss_distance_kilometers': {0: '29751820.03490958'}}

**Создаем цикл для работы с API с получением данных за определенное количество дней**

In [129]:
'''Структуры для работы.

items_list - список из данных по каждому объекту за определенный период
missed_colums_keys = [] - список для обработки словаря по одному объекту (удаление столбцов)
'''
items_list = []
missed_colums_keys = [
    'links','estimated_diameter',
    'close_approach_data',
    'relative_velocity',
    'miss_distance'
    ]

In [130]:
'''Создание списка заголовков

для требуемого пустого DataFrame
из DataFrame за один день
'''
key_list = list(df_item.to_dict())
key_list

['id',
 'name',
 'absolute_magnitude_h',
 'is_potentially_hazardous_asteroid',
 'is_sentry_object',
 'estimated_diameter_km_min',
 'estimated_diameter_km_max',
 'close_approach_date',
 'close_approach_date_full',
 'epoch_date_close_approach',
 'relative_velocity',
 'orbiting_body',
 'kilometers_per_second',
 'kilometers_per_hour',
 'miss_distance_kilometers']

In [131]:
# Создаем пустой DataFrame
df0 = pd.DataFrame(columns=key_list)

df0

,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_km_min,estimated_diameter_km_max,close_approach_date,close_approach_date_full,epoch_date_close_approach,relative_velocity,orbiting_body,kilometers_per_second,kilometers_per_hour,miss_distance_kilometers


In [132]:
# n - количество запросов(дней)
n = 100

In [133]:
''' Создаем цикл для получения данных

через API за заданный период времени
на основе алгоритма получения данных
за один день
'''


for i in range(1, n+1):
    current = get_url(url, 'next')
    url = current
    
    # Навигация по ссылкам, работаем с данными за один день
    url_level_0 = urllib.request.urlopen(url)
    data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
    
    # Cписок элементов за один день
    data_in_day = data_level_0.near_earth_objects[3]

    # Цикл по элементам списка за один день
    for data_day_item in data_in_day:
        dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
        dict_close_approach_data = data_day_item['close_approach_data'][0]
        dict_relative_velocity = dict_close_approach_data['relative_velocity']
        dict_miss_distance = dict_close_approach_data['miss_distance']

        # Чистим словарь от вложенных словарей
        for key in missed_colums_keys:
            data_day_item.pop(key, None)

        data_day_item_short = data_day_item
        
        # Создаем Dataframe из вложенных словарей
        df1 = pd.DataFrame([data_day_item_short])
        df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
        df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
        df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
        df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)
        
        # Чистим DataFrame от нерелевантных данных, переименовываем
        df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                        'neo_reference_id',
                                                        'miss_distance',
                                                        'miles_per_hour',
                                                        'astronomical',
                                                        'lunar',
                                                        'miles'
                                                        ], axis=1
        ).rename(columns={'kilometers': 'miss_distance_kilometers',
                        'estimated_diameter_min': 'estimated_diameter_km_min',
                        'estimated_diameter_max': 'estimated_diameter_km_max'})
        

        # Добавление в итоговый DataFrame данных построчно
        df0 = pd.concat([df0, df_item])

In [134]:
# Итоговый DataFrame.

# df0 = df0.head(15).reset_index()

df0

,index,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_km_min,estimated_diameter_km_max,close_approach_date,close_approach_date_full,epoch_date_close_approach,relative_velocity,orbiting_body,kilometers_per_second,kilometers_per_hour,miss_distance_kilometers
0,0,2416591,416591 (2004 LC2),18.66,True,False,0.492669,1.101641,2020-01-02,2020-Jan-02 01:53,1577929980000,NaN,Earth,18.5987778752,66955.6003508471,30662143.703405853
1,0,2506491,506491 (2003 UW29),20.77,True,False,0.186447,0.416908,2020-01-02,2020-Jan-02 09:35,1577957700000,NaN,Earth,37.6221532439,135439.7516780778,37818205.10274256
2,0,3591616,(2011 YP10),24.60,False,False,0.031956,0.071456,2020-01-02,2020-Jan-02 07:43,1577950980000,NaN,Earth,5.6403530723,20305.2710603878,10645283.689994342
3,0,3599868,(2012 DN31),24.00,False,False,0.042126,0.094198,2020-01-02,2020-Jan-02 13:28,1577971680000,NaN,Earth,21.6106016558,77798.1659609434,68102875.157882628
4,0,3623681,(2013 AW52),20.02,False,False,0.263363,0.588898,2020-01-02,2020-Jan-02 12:00,1577966400000,NaN,Earth,16.5306201679,59510.2326045044,27324640.191590356
5,0,3711166,(2015 DY53),21.43,False,False,0.137580,0.307638,2020-01-02,2020-Jan-02 14:40,1577976000000,NaN,Earth,8.3858595165,30189.0942594108,28943098.018751644
6,0,3736392,(2015 XZ168),24.70,False,False,0.030518,0.068240,2020-01-02,2020-Jan-02 16:33,1577982780000,NaN,Earth,9.0662034795,32638.3325260842,67642260.443873524
7,0,3760885,(2016 TR55),22.99,False,False,0.067074,0.149982,2020-01-02,2020-Jan-02 14:35,1577975700000,NaN,Earth,6.8362235494,24610.4047777292,67531035.773909354
8,0,3837605,(2019 AE3),27.40,False,False,0.008801,0.019681,2020-01-02,2020-Jan-02 16:08,1577981280000,NaN,Earth,8.2431067166,29675.1841799205,1865744.751847858
9,0,3943370,(2019 YH2),23.35,False,False,0.056827,0.127069,2020-01-02,2020-Jan-02 09:36,1577957760000,NaN,Earth,14.2787337462,51403.4414861465,2824432.603886633


In [135]:
df0.to_excel("NASA_Asteroids.xlsx",
             sheet_name='Sheet_1') 